In [42]:
# Dependencies
import tensorflow as tf
import pandas as pd
import numpy as np

In [43]:
print("\nTraining a neural network on Iris dataset using TensorFlow ")
print("Loading the Iris data to memory...")
# Loading the dataset
dataset = pd.read_csv('ourdata.csv')
print("Finish loading")


Training a neural network on Iris dataset using TensorFlow 
Loading the Iris data to memory...
Finish loading


In [44]:
# One-hot encoding for the categories
dataset = pd.get_dummies(dataset, columns=['POOLID']) 
values = list(dataset.columns.values)
print(dataset)

          SIZE    OFFSET    LENGTH  POOLID_24  POOLID_27  POOLID_60
0     0.000748  0.000000  0.027412          0          0          1
1     0.011206  0.000000  0.035966          0          0          1
2     0.013513  0.000000  0.036041          0          0          1
3     0.013576  0.000000  0.036050          0          0          1
4     0.022461  0.000000  0.026706          0          0          1
5     0.022461  0.000000  0.026737          0          0          1
6     0.022461  0.000000  0.026790          0          0          1
7     0.022461  0.000000  0.026811          0          0          1
8     0.022461  0.000000  0.026903          0          0          1
9     0.022461  0.000000  0.026939          0          0          1
10    0.022461  0.000000  0.026974          0          0          1
11    0.022461  0.000000  0.026978          0          0          1
12    0.022461  0.000000  0.027026          0          0          1
13    0.029738  0.750903  0.004619          0   

In [52]:
#preprocess the data 
target = np.array(dataset[values[-3:]], dtype='float32')
features = np.array(dataset[values[0:-3]], dtype='float32')

# Shuffle Data
indices = np.random.choice(len(features), len(features), replace=False)
X_values = features[indices]
y_values = target[indices]

# Creating a Train and a Test Dataset
test_size = 600
X_test = X_values[-test_size:]
X_train = X_values[:-test_size]
y_test = y_values[-test_size:]
y_train = y_values[:-test_size]

# print(X_train)
# print(X_test)
print(y_test)
# print(y_train)

[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [46]:
# define a neural network

# Initialize placeholders
X_data = tf.placeholder(shape=[None, 3], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 3], dtype=tf.float32)

#create seed for random_normal()
seed = 1234
np.random.seed(seed)
tf.set_random_seed(seed)

hidden_layer_nodes = 8
# We create a neural Network which contains 3 layers with 4, 8, 3 nodes repectively
w1 = tf.Variable(tf.random_normal(shape=[3,hidden_layer_nodes])) # Weight of the input layer
b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes]))   # Bias of the input layer
w2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes,3])) # Weight of the hidden layer
b2 = tf.Variable(tf.random_normal(shape=[3]))                    # Bias of the hidden layer
hidden_output = tf.nn.relu(tf.add(tf.matmul(X_data, w1), b1))
final_output = tf.nn.softmax(tf.add(tf.matmul(hidden_output, w2), b2))

# Loss Function
loss = tf.reduce_mean(-tf.reduce_sum(y_target * tf.log(final_output), axis=0))

# Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

print("A neural Network which contains 3 layers with 4, 8, 3 nodes repectively was created!")

A neural Network which contains 3 layers with 4, 8, 3 nodes repectively was created!


In [47]:
print('Training the model...')

# Interval / Epochs
interval = 100
epoch = 1000

# Initialize variables
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# Training the model...
for i in range(1, (epoch + 1)):
    sess.run(optimizer, feed_dict={X_data: X_train, y_target: y_train})
    if i % interval == 0:
        print('Epoch', i, '|', 'Loss:', sess.run(loss, feed_dict={X_data: X_train, y_target: y_train}))

print("Training finished\n")

Training the model...
Epoch 100 | Loss: 160.39238
Epoch 200 | Loss: 59.792347
Epoch 300 | Loss: 44.70972
Epoch 400 | Loss: 31.573957
Epoch 500 | Loss: 25.416616
Epoch 600 | Loss: 21.959908
Epoch 700 | Loss: 19.503973
Epoch 800 | Loss: 17.493502
Epoch 900 | Loss: 15.923043
Epoch 1000 | Loss: 14.898193
Training finished



In [55]:
# get the accuracy of the model
correct_prediction = tf.equal(tf.argmax(final_output, 1), tf.argmax(y_target,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("The accuracy of the model is:", sess.run(accuracy, feed_dict={X_data: X_test, y_target: y_test}))

The accuracy of the model is: 0.99666667


In [59]:
# Prediction
np.set_printoptions(precision=4)
unknown = np.array([[0.082286288, 0, 0.015198128]], dtype=np.float32)
predicted = sess.run(final_output, feed_dict={X_data: unknown})
# model.predict(unknown)
print("Using model to predict species for features: ", unknown)
print("\nPredicted softmax vector is: ",predicted)
Class_dict={'POOLID_24': 0, 'POOLID_27': 1, 'POOLID_60': 2}
species_dict = {v:k for k,v in Class_dict.items()}
print("\nPredicted species is: ", species_dict[np.argmax(predicted)])

Using model to predict species for features:  [[0.0823 0.     0.0152]]

Predicted softmax vector is:  [[6.2273e-07 3.7796e-06 1.0000e+00]]

Predicted species is:  POOLID_60
